In [13]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from prophet import Prophet

from sklearn.metrics import mean_squared_error, mean_absolute_error

plt.style.use('ggplot')
plt.style.use('fivethirtyeight')


In [14]:
# Load prepared dataset
df = pd.read_csv("../prepared_data/01_06_2021_to_20_06_2023_anais.csv", parse_dates=[0], index_col=[0])

In [15]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) /y_true)) * 100

In [16]:
color_palette = sns.color_palette('RdBu')
df['nb_cars'].plot(style='.', 
        figsize=(20, 8), 
        ms=5, 
        color=color_palette[0], 
        title="ENTRIES")

AttributeError: 'DataFrame' object has no attribute '_convert'

In [17]:
# CODE FIND ON KAGGLE TO USE IF WE WANT FEATURE CREATION
from pandas.api.types import CategoricalDtype
cat_type = CategoricalDtype(categories=['Monday','Tuesday',
                                        'Wednesday',
                                        'Thursday','Friday',
                                        'Saturday','Sunday'],
                            ordered=True)
def create_features(df, label=None):
    """
    Creates time series features from datetime index.
    """
    df = df.copy()
    df['date'] = df.index
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['weekday'] = df['date'].dt.day_name()
    df['weekday'] = df['weekday'].astype(cat_type)
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    df['date_offset'] = (df.date.dt.month*100 + df.date.dt.day - 320)%1300

    df['season'] = pd.cut(df['date_offset'], [0, 300, 602, 900, 1300], 
                          labels=['Spring', 'Summer', 'Fall', 'Winter']
                   )
    X = df[['hour','dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear','weekday',
           'season']]
    if label:
        y = df[label]
        return X, y
    return X

# X, y = create_features("DATAFRAME", label='MY_DATAFRAME')
# features_and_target = pd.concat([X, y], axis=1)

In [22]:
# TRAIN / TEST SPLIT

# Let's keep 2023 for the test split
split_date = '2023-01-01'
df_train = df.loc[df.index <= split_date].copy() 
df_test = df.loc[df.index > split_date].copy() 

# Rename 'nb_cars' in df_train and df_test
df_train = df_train.rename(columns={
    'nb_cars':'nb_cars_train', 
    'nb_cars_cxl': 'nb_cars_cxl_train',
    'nb_bookings': 'nb_bookings_train',
    'nb_bookings_cxl': 'nb_bookings_cxl_train'
    })
df_test = df_test.rename(columns={'nb_cars': 'nb_cars_test'})

# Join the two df
machin = df_test.join(df_train, how='outer')

#machin.plot(y=['nb_cars_test', 'nb_cars_train'], figsize=(20, 8), style='.', title="NB_CARS")

# # Let's see the result
# plt.show()

AttributeError: 'DataFrame' object has no attribute '_convert'

In [ ]:
# SIMPLE PROPHET MODEL
df_train_prophet = df_train.reset_index() \
    .rename(columns={'date':'ds',
                     'nb_cars_train':'y'})

In [ ]:
df_train_prophet.head()

In [ ]:
%%time
model = Prophet()
model.fit(df_train_prophet)

In [ ]:
install_name_tool -add_rpath @executable_path/cmdstan-2.26.1/stan/lib/stan_math/lib/tbb /Users/lefebvreremy/miniconda3/envs/env-carpark/lib/python3.10/site-packages/prophet/stan_model/prophet_model.bin